In [5]:
# %%
import os
import tempfile
import time
import re
import tkinter as tk
from tkinter import filedialog, messagebox
from datetime import datetime
from threading import Thread

import cv2
import numpy as np
import face_recognition
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

# Cargar modelo OCR una sola vez (puede tardar)
print("Cargando modelo OCR (docTR)...")
OCR_MODEL = ocr_predictor(pretrained=True)
print("Modelo OCR listo.")

C:\Users\Charlie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargando modelo OCR (docTR)...
Modelo OCR listo.


In [6]:
# %%
def eye_aspect_ratio(eye_points):
    p2_p6 = np.linalg.norm(np.array(eye_points[1]) - np.array(eye_points[5]))
    p3_p5 = np.linalg.norm(np.array(eye_points[2]) - np.array(eye_points[4]))
    p1_p4 = np.linalg.norm(np.array(eye_points[0]) - np.array(eye_points[3]))
    if p1_p4 == 0:
        return 0.0
    return (p2_p6 + p3_p5) / (2.0 * p1_p4)


def cargar_encoding_dni(dni_path):
    img = face_recognition.load_image_file(dni_path)
    encodings = face_recognition.face_encodings(img)
    if encodings:
        return encodings[0]
    scale = 2
    img_large = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    encodings = face_recognition.face_encodings(img_large, num_jitters=1)
    if encodings:
        return encodings[0]
    face_locs = face_recognition.face_locations(img_large, number_of_times_to_upsample=2, model="hog")
    if not face_locs:
        raise ValueError("No se detectó ningún rostro en la imagen del DNI.")
    encodings = face_recognition.face_encodings(img_large, known_face_locations=face_locs, num_jitters=1)
    if not encodings:
        raise ValueError("No se detectó ningún rostro en la imagen del DNI.")
    return encodings[0]


def indice_rostro_principal(face_locations):
    areas = []
    for i, (top, right, bottom, left) in enumerate(face_locations):
        areas.append((i, max(0, right - left) * max(0, bottom - top)))
    return max(areas, key=lambda item: item[1])[0]


def ratio_nariz_cara(landmarks):
    chin = landmarks["chin"]
    nose_tip = landmarks["nose_tip"]
    left_x = chin[0][0]
    right_x = chin[-1][0]
    nose_x = int(np.mean([p[0] for p in nose_tip]))
    width = max(1, right_x - left_x)
    return (nose_x - left_x) / width


def verificar_en_vivo(dni_path, cam_id=0, tolerance=0.5, timeout_seg=20):
    dni_encoding = cargar_encoding_dni(dni_path)
    cap = cv2.VideoCapture(cam_id, cv2.CAP_DSHOW)
    if not cap.isOpened():
        raise IOError("No se puede abrir la cámara.")
    blink_ok = False
    giro_izq_ok = False
    giro_der_ok = False
    vio_ojos_abiertos = False
    frames_ojos_cerrados = 0
    distancias_validas = []
    inicio = time.time()
    try:
        for _ in range(3):
            cap.read()
        while (time.time() - inicio) < timeout_seg:
            ret, frame = cap.read()
            if not ret:
                continue
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb, model="hog")
            if not face_locations:
                cv2.putText(frame, "No se detecta rostro", (20, 35),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                cv2.imshow("Verificacion en vivo", frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                continue
            idx = indice_rostro_principal(face_locations)
            loc = [face_locations[idx]]
            encs = face_recognition.face_encodings(rgb, known_face_locations=loc)
            lms = face_recognition.face_landmarks(rgb, face_locations=loc)
            if not encs or not lms:
                continue
            cam_encoding = encs[0]
            landmarks = lms[0]
            if "left_eye" in landmarks and "right_eye" in landmarks:
                ear_left = eye_aspect_ratio(landmarks["left_eye"])
                ear_right = eye_aspect_ratio(landmarks["right_eye"])
                ear = (ear_left + ear_right) / 2.0
                if ear > 0.23:
                    if vio_ojos_abiertos and frames_ojos_cerrados >= 2:
                        blink_ok = True
                    vio_ojos_abiertos = True
                    frames_ojos_cerrados = 0
                elif ear < 0.19:
                    frames_ojos_cerrados += 1
            if "chin" in landmarks and "nose_tip" in landmarks:
                ratio = ratio_nariz_cara(landmarks)
                if ratio < 0.42:
                    giro_izq_ok = True
                if ratio > 0.58:
                    giro_der_ok = True
            liveness_ok = blink_ok and giro_izq_ok and giro_der_ok
            if liveness_ok:
                distancia = face_recognition.face_distance([dni_encoding], cam_encoding)[0]
                distancias_validas.append(float(distancia))
            cv2.putText(frame,
                        f"Liveness:{'OK' if liveness_ok else 'PENDIENTE'} Blink:{blink_ok}",
                        (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame,
                        f"GiroIzq:{giro_izq_ok} GiroDer:{giro_der_ok}",
                        (20, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame,
                        "Parpadea y gira la cabeza a ambos lados (Q para salir)",
                        (20, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 2)
            cv2.imshow("Verificacion en vivo", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            if len(distancias_validas) >= 6:
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()
    if not (blink_ok and giro_izq_ok and giro_der_ok):
        raise ValueError("Error: prueba de vida no superada.")
    if not distancias_validas:
        raise ValueError("Error: no se pudo obtener comparación válida con el DNI.")
    distancia_final = float(np.median(distancias_validas))
    coincide = distancia_final <= tolerance
    return coincide, distancia_final

In [7]:
# %%
def extraer_texto_ocr(imagen_path, confidence=0.5):
    """Ejecuta OCR sobre una imagen y devuelve lista de líneas de texto."""
    try:
        doc = DocumentFile.from_images(imagen_path)
        result = OCR_MODEL(doc)
        lineas = []
        for page in result.pages:
            for block in page.blocks:
                for line in block.lines:
                    palabras = [word.value for word in line.words if word.confidence > confidence]
                    if palabras:
                        lineas.append(" ".join(palabras))
        return lineas
    except Exception as e:
        raise RuntimeError(f"Error en OCR de {imagen_path}: {str(e)}")


def validar_datos_dni(anverso_path, reverso_path):
    """
    Valida los datos del DNI comparando anverso y MRZ del reverso.
    Retorna (exito, mensaje, detalles) donde detalles es un dict con los campos.
    """
    hoy = datetime.now()
    try:
        # OCR anverso y reverso
        f_lines = extraer_texto_ocr(anverso_path, 0.5)
        b_lines = extraer_texto_ocr(reverso_path, 0.4)
        f_all = " ".join(f_lines).upper()

        # Buscar líneas candidatas a MRZ en el reverso
        m_c = [l.replace(" ", "") for l in b_lines if len(l.replace(" ", "")) >= 28 and ("<" in l or "ESP" in l)]
        if len(m_c) < 3:
            return False, "No se detectaron las 3 líneas del MRZ en el reverso.", {}

        # Identificar las tres líneas del MRZ (asumiendo orden)
        l1 = next((l for l in m_c if "ESP" in l[:10]), m_c[0])
        idx = m_c.index(l1)
        l2 = m_c[idx+1] if len(m_c) > idx+1 else m_c[1]
        l3 = m_c[idx+2] if len(m_c) > idx+2 else m_c[-1]

        # Parsear campos del MRZ (estructura típica DNI español)
        m_data = {
            "id": l1[5:14].replace("<", ""),                     # IDESP
            "dni": l1[15:23].replace("<", ""),                   # Número DNI
            "s": l2[7],                                           # Sexo
            "b": f"{l2[4:6]} {l2[2:4]} 20{l2[0:2]}",            # Fecha nacimiento
            "e": f"{l2[12:14]} {l2[10:12]} 20{l2[8:10]}",        # Fecha caducidad
            "n": l3.replace("<", " ").strip()                     # Nombre
        }

        # Extraer fechas del anverso
        dates = re.findall(r'\d{2} \d{2} \d{4}', f_all)
        # La fecha de emisión suele ser la tercera fecha (no nacimiento ni caducidad)
        iss = next((d for d in dates if d != m_data["b"] and d != m_data["e"]), "N/A")

        # Función para validar fecha respecto a hoy
        def fecha_valida(fecha_str, debe_ser_pasada):
            try:
                d = datetime.strptime(fecha_str.replace(" ", ""), "%d%m%Y")
                return (d < hoy) if debe_ser_pasada else (d > hoy)
            except:
                return False

        # Validar fechas
        emision_ok = fecha_valida(iss, debe_ser_pasada=True) if iss != "N/A" else False
        caducidad_ok = fecha_valida(m_data["e"], debe_ser_pasada=False)

        # Comparar campos con el anverso
        dni_f = (re.search(r'\d{8}[A-Z]', f_all) or re.search(r'\d{8}', f_all)).group(0) if re.search(r'\d{8}', f_all) else "N/A"
        id_f = re.search(r'[A-Z]{3}\d{6}', f_all).group(0) if re.search(r'[A-Z]{3}\d{6}', f_all) else "N/A"

        dni_ok = (dni_f[:8] == m_data["dni"]) if dni_f != "N/A" else False
        id_ok = (id_f == m_data["id"]) if id_f != "N/A" else False
        sexo_ok = m_data["s"] in f_all
        # Nombre: al menos una palabra significativa del nombre aparece en anverso
        palabras_nombre = [p for p in m_data["n"].replace("K", " ").split() if len(p) > 3]
        nombre_ok = any(p in f_all for p in palabras_nombre) if palabras_nombre else False

        # Coherencia global
        datos_ok = dni_ok and id_ok and sexo_ok and nombre_ok and emision_ok and caducidad_ok

        detalles = {
            "dni": (m_data["dni"], dni_f, dni_ok),
            "id_esp": (m_data["id"], id_f, id_ok),
            "sexo": (m_data["s"], "detectado" if sexo_ok else "no", sexo_ok),
            "nacimiento": (m_data["b"], iss if iss in dates else "N/A", emision_ok),  # Nota: nacimiento vs emisión
            "caducidad": (m_data["e"], "N/A", caducidad_ok),
            "nombre": (m_data["n"], "detectado" if nombre_ok else "no", nombre_ok),
            "emision": (iss, "N/A", emision_ok)
        }

        if datos_ok:
            return True, "Datos del DNI válidos y vigentes.", detalles
        else:
            return False, "Algunos datos del DNI no coinciden o están fuera de vigencia.", detalles

    except Exception as e:
        return False, f"Error durante la validación: {str(e)}", {}

In [8]:
# %%
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Verificación de Identidad Completa")
        self.root.geometry("500x400")
        self.root.resizable(False, False)

        self.anverso_path = None
        self.reverso_path = None

        # Variables de estado
        self.estado_anverso = tk.StringVar(value="[PENDIENTE] No cargado")
        self.estado_reverso = tk.StringVar(value="[PENDIENTE] No cargado")

        self.create_widgets()

    def create_widgets(self):
        # Título
        tk.Label(self.root, text="Verificación de Identidad", font=("Arial", 16, "bold")).pack(pady=10)

        # Frame para anverso
        frame_anverso = tk.LabelFrame(self.root, text="Anverso del DNI", padx=10, pady=10)
        frame_anverso.pack(fill="x", padx=20, pady=5)

        tk.Button(frame_anverso, text="Cargar imagen", command=lambda: self.cargar_imagen("anverso"), width=15).pack(side="left", padx=5)
        tk.Button(frame_anverso, text="Tomar foto", command=lambda: self.tomar_foto("anverso"), width=15).pack(side="left", padx=5)
        tk.Label(frame_anverso, textvariable=self.estado_anverso, fg="blue").pack(side="left", padx=10)

        # Frame para reverso
        frame_reverso = tk.LabelFrame(self.root, text="Reverso del DNI", padx=10, pady=10)
        frame_reverso.pack(fill="x", padx=20, pady=5)

        tk.Button(frame_reverso, text="Cargar imagen", command=lambda: self.cargar_imagen("reverso"), width=15).pack(side="left", padx=5)
        tk.Button(frame_reverso, text="Tomar foto", command=lambda: self.tomar_foto("reverso"), width=15).pack(side="left", padx=5)
        tk.Label(frame_reverso, textvariable=self.estado_reverso, fg="blue").pack(side="left", padx=10)

        # Botón de verificación
        self.btn_verificar = tk.Button(self.root, text="Iniciar verificación completa",
                                       command=self.iniciar_verificacion, state=tk.DISABLED,
                                       bg="lightblue", font=("Arial", 12), padx=10, pady=5)
        self.btn_verificar.pack(pady=15)

        # Barra de estado
        self.status_var = tk.StringVar(value="Listo. Cargue ambas caras del DNI.")
        self.status_label = tk.Label(self.root, textvariable=self.status_var, bg="#e5e7eb", font=("Arial", 9))
        self.status_label.pack(fill="x", side="bottom")

    def cargar_imagen(self, lado):
        file_path = filedialog.askopenfilename(
            title=f"Seleccionar imagen del {lado}",
            filetypes=[("Archivos de imagen", "*.jpg *.jpeg *.png *.bmp *.tiff")]
        )
        if file_path:
            self._asignar_imagen(lado, file_path)

    def tomar_foto(self, lado):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            messagebox.showerror("Error", "No se pudo abrir la cámara.")
            return

        cv2.namedWindow(f"Capturar {lado} - ESPACIO para tomar, ESC cancelar")
        img_capturada = None

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            cv2.putText(frame, "ESPACIO para capturar, ESC para cancelar", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.imshow(f"Capturar {lado}", frame)
            key = cv2.waitKey(1) & 0xFF
            if key == 27:
                break
            elif key == 32:
                img_capturada = frame.copy()
                break

        cap.release()
        cv2.destroyAllWindows()

        if img_capturada is not None:
            temp_dir = tempfile.gettempdir()
            temp_path = os.path.join(temp_dir, f"dni_{lado}_{int(time.time())}.jpg")
            cv2.imwrite(temp_path, img_capturada)
            self._asignar_imagen(lado, temp_path)

    def _asignar_imagen(self, lado, ruta):
        if lado == "anverso":
            self.anverso_path = ruta
            self.estado_anverso.set(f"[OK] {os.path.basename(ruta)}")
        else:
            self.reverso_path = ruta
            self.estado_reverso.set(f"[OK] {os.path.basename(ruta)}")

        # Habilitar botón si ambas cargas están listas
        if self.anverso_path and self.reverso_path:
            self.btn_verificar.config(state=tk.NORMAL)
            self.status_var.set("Ambas caras cargadas. Pulse 'Iniciar verificación'.")
        else:
            self.btn_verificar.config(state=tk.DISABLED)

    def iniciar_verificacion(self):
        if not self.anverso_path or not self.reverso_path:
            messagebox.showwarning("Advertencia", "Debe cargar ambas caras del DNI.")
            return

        # Deshabilitar botón durante el proceso
        self.btn_verificar.config(state=tk.DISABLED)
        self.status_var.set("Validando datos del DNI... (puede tardar unos segundos)")

        # Ejecutar validación en un hilo para no bloquear la UI
        Thread(target=self._proceso_validacion_y_biometria, daemon=True).start()

    def _proceso_validacion_y_biometria(self):
        try:
            # Paso 1: Validar datos del DNI (MRZ)
            exito, mensaje, detalles = validar_datos_dni(self.anverso_path, self.reverso_path)

            if not exito:
                self.root.after(0, lambda: self._mostrar_error_validacion(mensaje))
                return

            # Si los datos son válidos, mostrar resumen y preguntar si continuar
            resumen = self._formatear_resumen(detalles)
            continuar = messagebox.askyesno("Datos válidos",
                                            f"Los datos del DNI son correctos y vigentes.\n\n{resumen}\n\n¿Desea continuar con la verificación biométrica?")
            if not continuar:
                self.root.after(0, lambda: self._finalizar_proceso(False, "Verificación cancelada por el usuario."))
                return

            # Paso 2: Verificación biométrica en vivo
            self.root.after(0, lambda: self.status_var.set("Iniciando verificación en vivo... (mire a la cámara)"))
            coincide, distancia = verificar_en_vivo(
                dni_path=self.anverso_path,  # Usamos el anverso para el rostro
                cam_id=0,
                tolerance=0.5,
                timeout_seg=20
            )

            if coincide:
                self.root.after(0, lambda: self._finalizar_proceso(True,
                    f"Verificación exitosa.\nCoincidencia biométrica con distancia: {distancia:.4f}"))
            else:
                self.root.after(0, lambda: self._finalizar_proceso(False,
                    f"La persona no coincide con el DNI (distancia: {distancia:.4f})."))

        except Exception as e:
            self.root.after(0, lambda: self._mostrar_error_validacion(f"Error inesperado: {str(e)}"))

    def _formatear_resumen(self, detalles):
        lineas = []
        for campo, (valor_mrz, valor_front, ok) in detalles.items():
            estado = "[OK]" if ok else "[ERROR]"
            if campo == "emision":
                lineas.append(f"Emisión: {valor_mrz} {estado}")
            elif campo == "caducidad":
                lineas.append(f"Caducidad: {valor_mrz} {estado}")
            elif campo == "nombre":
                lineas.append(f"Nombre: {valor_mrz[:30]}... {estado}")
            elif campo == "dni":
                lineas.append(f"DNI: {valor_mrz} (frontal: {valor_front}) {estado}")
            elif campo == "id_esp":
                lineas.append(f"IDESP: {valor_mrz} (frontal: {valor_front}) {estado}")
            elif campo == "sexo":
                lineas.append(f"Sexo: {valor_mrz} {estado}")
            elif campo == "nacimiento":
                lineas.append(f"Nacimiento: {valor_mrz} {estado}")
        return "\n".join(lineas)

    def _mostrar_error_validacion(self, mensaje):
        messagebox.showerror("Error de validación", mensaje)
        self.status_var.set("Validación fallida. Revise las imágenes.")
        self.btn_verificar.config(state=tk.NORMAL)

    def _finalizar_proceso(self, exito, mensaje):
        if exito:
            messagebox.showinfo("Éxito", mensaje)
            self.status_var.set("Verificación completada con éxito.")
        else:
            messagebox.showerror("Error", mensaje)
            self.status_var.set("Verificación fallida.")
        self.btn_verificar.config(state=tk.NORMAL)

In [ ]:
# %%
if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()